In [235]:
#%load_ext tensorboard
import numpy as np
#!pip install torch torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import pandas as pd

from PIL import Image, ImageFile

import matplotlib.pyplot as plt

import torchvision.models as models

from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split

import os
import getpass
import sys

In [2]:
# X = np.asarray([[0,1],[1,1],[4,0],[5,0],[6,0],[7,0],[8,1],[9,0],[10,0]])
# X_ = X.T
# X_train, y_train = train_test_split(X, test_size=0.33,stratify = X_[1])
# print(X_train)
# print(y_train)

In [3]:
torch.cuda.get_device_name(0)

'GeForce GTX TITAN X'

In [76]:
username = getpass.getuser()

train_path = f'/home/users/{username}/kaggle-data/jpeg/train'
train_meta = f'/home/users/{username}/kaggle-data/train.csv'

test_path = f'/home/users/{username}/kaggle-data/jpeg/test'
test_meta = f'/home/users/{username}/kaggle-data/test.csv'



classes = 2

In [101]:
class KaggleDataset(torch.utils.data.Dataset):
    
    def __init__(self, img_path, meta_path, test_path, test_meta_path,tr = True, test= False):
        super(KaggleDataset, self).__init__()
        self.img_path = img_path
        self.meta_path = meta_path
        self.meta = pd.read_csv(self.meta_path)
        self.tr = tr
        self.test = test
        self.train, self.val = train_test_split(self.meta, test_size = 0.2, stratify = self.meta.target)
        if test:
            self.test_path = test_path
            self.test_meta_path = test_meta_path
            self.test_meta = pd.read_csv(self.test_meta_path)
            
    def __len__(self):
        if self.tr and not self.test:
            return len(self.train)
        elif not self.tr and not self.test:
            return len(self.val)
        else: 
            return len(self.test_meta)
    
    def __getitem__(self, idx):
        
        if self.tr and not self.test:
            label = self.train.iloc()[idx]

            target = label.target

            path = os.path.join(self.img_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)
            return input_tensor, target
        
        elif not self.tr and not self.test:
            label = self.val.iloc()[idx]

            target = label.target

            path = os.path.join(self.img_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)
            return input_tensor, target
        
        else:
            label = self.test_meta.iloc()[idx]

#             target = self.label.index(meta.target)

            path = os.path.join(self.test_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)
            
            return input_tensor, label.image_name#self.test_meta['image_name'].tolist()

In [143]:
# self.train, self.val = train_test_split(meta, test_size = 0.2, stratify = meta.target)
def get_dataset(img_path, meta_path,test_path, test_meta_path, train, test):
    dataset = KaggleDataset(img_path, meta_path,test_path, test_meta_path, train, test)
    return dataset

def get_loader(dataset, shuffle, size=128):
    loader = DataLoader(dataset, batch_size=size,
                        shuffle=shuffle, num_workers=8)
    return loader

def build_dataloader(img_path, meta_path,test_path, test_meta_path,):
 
    train_dataset = get_dataset(img_path, meta_path,test_path, test_meta_path,
                                True, False)
    val_dataset = get_dataset(img_path, meta_path,test_path, test_meta_path,
                              False, False)
    test_dataset = get_dataset(img_path, meta_path,test_path, test_meta_path,
                               True, True)
    
    train_loader = get_loader(train_dataset, True)
    val_loader = get_loader(val_dataset, False)
    test_loader = get_loader(test_dataset, False)
    
    
    return train_loader, val_loader, test_loader


In [103]:
# resnet.fc
# Linear(in_features=2048, out_features=1000, bias=True)

In [165]:
class Resnet(pl.LightningModule):

    def __init__(self, freeze=True, lr=0.01, names=None):
        super(Resnet, self).__init__()
        self.resnet = models.resnext50_32x4d(pretrained=True)
        ct = 0
        if freeze is True:
            # freeze feature layers
            for child in self.resnet.children():
                ct+=1
                if ct<10:
                    for param in child.parameters():
                        param.requires_grad = False
        
        # adjust classes
        self.resnet.fc = nn.Linear(2048, 2)
        
        self.loaders = None
        self.lr = lr
        self.names = names
        
    def set_names(self, names):
        self.names = names
    def forward(self, inp):
        return self.resnet(inp)
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        tensorboard_logs = {'train_loss' : loss}
        return {'loss': loss, 'log': tensorboard_logs}
    
    def configure_optimizers(self):
        optimizer = SGD(self.parameters(),
                       lr=self.lr,
                       momentum=0.9,
                       weight_decay=0.0005)
        scheduler = StepLR(optimizer, 1)
        return [optimizer], [scheduler]
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return {'val_loss': loss}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        x, y = batch
        y_hat = self(x).argmax(dim=1)
        acc = (y == y_hat).float().mean()
#         c_accs = []
#         for i in range(classes):
#             class_elem = y == i
#             c_acc = (y_hat[class_elem] == i).sum().float() / (class_elem.sum() + 1e-9)
#             c_accs.append(c_acc)
        return {'accuracy' : acc}#, 'class_accuracies' : c_accs}
        #return y_hat

    def test_epoch_end(self, outputs):
        avg_acc = torch.stack([x['accuracy'] for x in outputs]).mean().item()
        #c_accs = [x['class_accuracies'] for x in outputs]
        #avg_c_acc = [torch.stack([l[i] for l in c_accs]).mean().item() for i in range(classes)]
#         if self.names is not None:
#             names = pd.read_pickle(self.names)
#             names = names[0].to_list()
#             avg_c_acc = list(zip(names, avg_c_acc))
        return {'accuracy' : avg_acc}#, 'class_accuracies': avg_c_acc}
    
    def build_loaders(self):
        self.loaders = \
            build_dataloader(train_path, train_meta, test_path, test_meta)
    
    def train_dataloader(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[0]
    
    def val_dataloader(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[1]
    
    def test_dataloader(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[1]
    
    def test_data(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[2]
    

In [166]:
def run_training(version='v1'):
    checkpoint_callback = ModelCheckpoint(
        filepath='models/resnext50.ckpt',
        verbose=True,
    )

    trainer = pl.Trainer( 
        checkpoint_callback=checkpoint_callback,
        gpus=[0],
        distributed_backend='dp', 
        max_nb_epochs=3,
        val_check_interval=0.25,
    )
    
    if version == 'v1':
        model = Resnet()
    else:
        model = Resnet(freeze=False, lr=0.001)
        model.load_state_dict(torch.load('resnext50.pt'))

    trainer.fit(model)

    torch.save(model.state_dict(), f'resnext50{version}.pt')

In [167]:
# # if version == 'v1':
# #         model = Resnet()
# #     else:
# # checkpoint_callback = ModelCheckpoint(
# #         filepath='models/_ckpt_epoch_2.ckpt',
# #         verbose=True,
# #     )
# #model = Resnet()#.load_from_checkpoint(checkpoint_path="models/_ckpt_epoch_2.ckpt")
# # trainer = pl.Trainer( 
# #     checkpoint_callback=checkpoint_callback,
# #     gpus=[0],
# #     distributed_backend='dp', 
# #     max_nb_epochs=1,
# #     val_check_interval=0.25,
# # )
# trainer = pl.Trainer(resume_from_checkpoint='models/_ckpt_epoch_2.ckpt',
#                         gpus=[0],
#                         distributed_backend='dp', 
#                         max_nb_epochs=1,
#                         val_check_interval=0.25)
# #model = Resnet(freeze=False, lr=0.001)
# #model.load_state_dict(torch.load('models/resnext50_32x4dv1.pt'))
# model = Resnet()
# trainer.fit(model)

# torch.save(model.state_dict(), 'models/resnext50_32x4dv1.ckpt')

File "isic_notebook.py", line 309, in run_training
    torch.save(model.state_dict(), f'checkpoints/resnext50_32x4d{version}.pt')
  File "/home/users/lameski/.local/lib/python3.6/site-packages/torch/serialization.py", line 369, in save
    with _open_file_like(f, 'wb') as opened_file:
  File "/home/users/lameski/.local/lib/python3.6/site-packages/torch/serialization.py", line 234, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/users/lameski/.local/lib/python3.6/site-packages/torch/serialization.py", line 215, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/resnext50_32x4dv1.pt'

In [168]:
def evaluate():
    trainer = pl.Trainer(
        gpus=[0],
        distributed_backend='dp')
    model = Resnet()
    #img, target = model.test_dataloader()
    #model.set_names(target)
    model.load_state_dict(torch.load('models/resnext50_32x4dv1.ckpt'))

    trainer.test(model, model.test_dataloader())

In [169]:
evaluate()

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
TEST RESULTS
{'accuracy': 0.9826763868331909}
--------------------------------------------------------------------------------



In [170]:
class PredictDataset(torch.utils.data.Dataset):

    def __init__(self, test_path, test_meta_path):
        super(PredictDataset, self).__init__()

        self.test_path = test_path
        self.test_meta_path = test_meta_path
        self.test_meta = pd.read_csv(self.test_meta_path)

    def __len__(self):
        return len(self.test_meta)

    def __getitem__(self, idx):

            label = self.test_meta.iloc()[idx]

            path = os.path.join(self.test_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)

            return input_tensor 

In [208]:
def build_predict_dataloader(img_path, meta_path):
    dataset = PredictDataset(img_path, meta_path)
    loader = get_loader(dataset, False, size=1)
    data_names = pd.read_csv(meta_path)
    data_names = data_names.image_name
    return (loader, data_names.to_numpy())

In [209]:
a,b = build_predict_dataloader(test_path, test_meta)
len(b)

10982

In [211]:
def predict():
    device = torch.device('cuda:0')
    cpu = torch.device('cpu')
    model = Resnet('/models/resnext50_32x4d.pt')
    model.to(device)
    loader, data_names = build_predict_dataloader(test_path, test_meta)
    #loader = model.test_data()
    model.eval()
    preds = []
    
    with torch.no_grad():
        i = 0
        for img in tqdm(loader):
            img = img.to(device)
            #len(img)
            #break
            pred = model(img).argmax(dim=1)
            
            #for j in range(64):
            preds.append([data_names[i],pred.to(cpu)])
            i+=1
    #data = [preds[i] for i in range(len(preds))]
    df = pd.DataFrame(preds, columns=['image_name', 'target'])
    df.to_csv('submission_.csv') 
    
    

In [212]:
#run_training()
predict()


  3%|▎         | 384/10982 [00:28<13:04, 13.51it/s]


  7%|▋         | 772/10982 [00:55<10:48, 15.75it/s]


 11%|█         | 1180/10982 [01:25<12:35, 12.98it/s]


 15%|█▍        | 1595/10982 [01:53<11:48, 13.24it/s]


 18%|█▊        | 2006/10982 [02:21<09:52, 15.15it/s]


...

 89%|████████▊ | 9736/10982 [11:26<01:18, 15.82it/s]


 92%|█████████▏| 10130/10982 [11:55<01:31,  9.31it/s]


 96%|█████████▌| 10521/10982 [12:22<00:23, 19.39it/s]


 99%|█████████▉| 10927/10982 [12:50<00:04, 13.74it/s]


100%|██████████| 10982/10982 [12:53<00:00, 14.20it/s]


In [220]:
import re
df = pd.read_csv("submission_.csv")
suma = 0
for i in df.target:
    suma+=int(i[8:-2])
suma
# names = []
# targets = []
# for i in range(len(df)):
# #     temp = df.image_name[i][2:-2].split("', '")
#     for n in temp:
#         names.append(n)
#     temp = re.split(', |,\n        ',df.target[i][8:-2])
#     for t in temp:
#         targets.append(t)

# output = pd.DataFrame(list(zip(names, targets)), columns=['image_name', 'target'])
# output.to_csv("submission_output3.csv")
# output

10956

In [221]:
device = torch.device('cuda:0')
cpu = torch.device('cpu')
model = Resnet('/models/resnext50_32x4d.pt')
model.to(device)
loader, data_names = build_predict_dataloader(test_path, test_meta)
#loader = model.test_data()
model.eval()


In [237]:
preds = []
with torch.no_grad():
    i = 0
    for img in tqdm(loader):
        img = img.to(device)
        #len(img)
        #break
        pred = model(img).argmax(dim=1).to(cpu).numpy()[0]

        #for j in range(64):
        preds.append([data_names[i],pred])
        if pred==1:
            print("Element num:%d" % i)
        i+=1

Element num:110
Element num:156
Element num:203
Element num:215
Element num:296
Element num:464
Element num:485
Element num:494
Element num:528
Element num:544
Element num:576
Element num:579
Element num:769
Element num:859
Element num:887
Element num:1218
Element num:1399
Element num:1447
Element num:1508
Element num:1533
Element num:1601
Element num:1709
Element num:1729
Element num:1799
Element num:1828
Element num:1983
Element num:2015
Element num:2052
Element num:2059
Element num:2230
Element num:2252
Element num:2342
Element num:2367
Element num:2446
Element num:2466
Element num:2561
Element num:2569
Element num:2739
Element num:2839
Element num:2893
Element num:2944
Element num:2959
Element num:3159
Element num:3164
Element num:3178
Element num:3179
Element num:3206
Element num:3305
Element num:3317
Element num:3351
Element num:3415
Element num:3529
Element num:3562
Element num:3596
Element num:3708
Element num:3757
Element num:3770
Element num:3818
Element num:4010
Element num:

In [238]:
df = pd.DataFrame(preds, columns=['image_name', 'target'])
df.to_csv('submission_.csv')